In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt, cm
from IPython import get_ipython
ipy = get_ipython()
if ipy is not None:
    ipy.run_line_magic('matplotlib', 'inline')

In [3]:
road = cv2.imread('D:\Dev\OpenCV\images\image_road.jpg')

In [4]:
# read_copy = road.copy()
road_copy = np.copy(road)

In [5]:
# plt.imshow(road_copy)

In [6]:
road.shape[:2]

(600, 800)

In [7]:
marker_image = np.zeros(road.shape[:2], dtype=np.int32)

In [8]:
segments = np.zeros(road.shape, dtype=np.uint8)

In [9]:
marker_image.shape

(600, 800)

In [10]:
segments.shape

(600, 800, 3)

In [11]:
cm.tab10(0)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)

In [12]:
def create_rgb(i):
    return tuple(np.array(cm.tab10(i)[:3])*255)

In [13]:
colors = []
for i in range(10):
    colors.append(create_rgb(i))

In [14]:
colors

[(31.0, 119.0, 180.0),
 (255.0, 127.0, 14.0),
 (44.0, 160.0, 44.0),
 (214.0, 39.0, 40.0),
 (148.0, 103.0, 189.0),
 (140.0, 86.0, 75.0),
 (227.0, 119.0, 194.0),
 (127.0, 127.0, 127.0),
 (188.0, 189.0, 34.0),
 (23.0, 190.0, 207.0)]

In [15]:
# GLOBAL VARIABLES
# COLOR CHOICE
n_markers = 10  # [0 to 9]
current_marker = 1
# MARKERS UPDATED BY WATERSHED
marks_updated = False


# CALLBACK Function
def mouse_callback(event, x, y, flags, param):
    global marks_updated
    if event == cv2.EVENT_LBUTTONDOWN:
        # MARKERS PASSED TO THE WATERSHED ALGORITHM
        cv2.circle(marker_image, (x, y), 10, current_marker, -1)
        # USER SEES ON THE ROAD IMAGE
        cv2.circle(road_copy, (x, y), 10, colors[current_marker], -1)
        
        marks_updated = True

In [17]:
cv2.namedWindow('Road Image')
cv2.setMouseCallback('Road Image', mouse_callback)

# LOOP
while True:
    cv2.imshow('Watershed Segments', segments)
    cv2.imshow('Road Image', road_copy)
    
    # CLOSE ALL WINDOW
    k = cv2.waitKey(1)
    if k == 27:
        break
    # CLEARING ALL THE COLORS PRESS C KEY
    elif k == ord('c'):
        road_copy = road.copy()
        marker_image = np.zeros(road.shape[:2], dtype=np.int32)
        segments = np.zeros(road.shape, dtype=np.uint8)
    # UPDATE COLOR CHOICE
    elif k > 0 and chr(k).isdigit():
        current_marker = int(chr(k))
    
    # UPDATE THE MARKING
    if marks_updated:
        marker_image_copy = marker_image.copy()
        cv2.watershed(road, marker_image_copy)
        
        segments = np.zeros(road.shape, dtype=np.uint8)
        for color_index in range(n_markers):
            # COLORING SEGMENTS, NUMPY CALL
            segments[marker_image_copy == color_index] = colors[color_index]


cv2.destroyAllWindows()